# Введение в обработку текста на естественном языке

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Введение в обработку текста на естественном языке"
* https://www.nltk.org/api/nltk.metrics.distance.html
* https://pymorphy2.readthedocs.io/en/stable/user/guide.html
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/noble6/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. При помощи расстояния Левенштейна иправьте опечатку в слове "велечайшим".

In [ ]:
fil

2. Разбейте текст из формулировки второго задания на слова. Проведите стемминг и лемматизацию слов.

In [2]:
from nltk.stem import SnowballStemmer
from nltk import word_tokenize
import pymorphy2

In [3]:
text = '''Разбейте текст из формулировки второго задания на слова. Проведите стемминг и лемматизацию слов.'''

3. Преобразуйте предложения из формулировки задания 2 в векторы при помощи `CountVectorizer`. Выведите на экран словарь обученного токенизатора.

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk import sent_tokenize

In [5]:
text = '''Разбейте текст из формулировки второго задания на слова. Проведите стемминг и лемматизацию слов.'''

## Лабораторная работа 7

1\. Загрузите данные из файла `ru_recipes_sample.csv` в виде `pd.DataFrame` `recipes` Используя регулярные выражения, удалите из описаний (столбец `description`) все символы, кроме русских букв, цифр и пробелов. Приведите все слова в описании к нижнему регистру. Сохраните полученный результат в столбец `description`.

In [6]:
import pandas as pd
import re

recipes = pd.read_csv("07_nlp_data/ru_recipes_sample.csv")

#recipes["description"] = recipes["description"].apply(lambda x: x if re.search(r"\w\w", x) else None)
recipes["description"] = recipes["description"].replace(to_replace=r"[^А-Яа-яЁё0-9\s]", value="", regex=True)
recipes["description"] = recipes["description"].apply(lambda x: x.lower())

recipes["description"]

0       этот коктейль готовлю из замороженной клубники...
1                                         быстро и вкусно
2                сытный овощной салатик пальчики оближете
3       картофельное пюре и куриные котлеты  вкусная к...
4       вишневая наливка имеет яркий вишневый вкус кот...
                              ...                        
3462    для тех кто любит чечевицу вам сюда очень вкус...
3463    баклажановые фантазии продолжаются предлагаю в...
3464    мое любимое блюдо лазанья но кушать только фар...
3465    прошлым летом варила варенье из одуванчиков по...
3466     и три корочки хлеба  сделал заказ буратино в ...
Name: description, Length: 3467, dtype: object

### Расстояние редактирования

2\. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние Левенштейна. Выведите на экран результат в следующем виде:

```
d(word1, word2) = x
```

In [7]:
unique_words = list(set(nltk.word_tokenize(" ".join(recipes['description']).replace('\r', ' ').replace('\n', ' '))))

unique_words[:10]

['серым',
 'светланка',
 'осталась',
 'случиться',
 'овсяными',
 'острой',
 'догадается',
 'ошеломительным',
 'великолепно',
 'девиз']

In [8]:
import random

random_words = random.sample(unique_words, k=10)

for i in range(0, len(random_words) - 1, 2):
    print(f"d({random_words[i]}, {random_words[i+1]}) = {nltk.edit_distance(random_words[i], random_words[i+1])}")

d(горячего, готовке) = 6
d(ямальская, кушать) = 8
d(мечта, запомню) = 7
d(разлетелись, фальшивыми) = 9
d(вашей, представителя) = 11


3\. Напишите функцию, которая принимает на вход 2 текстовые строки `s1` и `s2` и при помощи расстояния Левенштейна определяет, является ли строка `s2` плагиатом `s1`. Функция должна реализовывать следующую логику: для каждого слова `w1` из `s1` проверяет, есть в `s2` хотя бы одно слово `w2`, такое, что расстояние Левенштейна между `w1` и `w2` меньше 2, и считает количество таких слов в `s1` $P$. 

$$ P = \#\{w_1 \in s_1\ | \exists w_2 \in s_2 : d(w_1, w_2) < tol\}$$

$$ L = max(|s1|, |s2|) $$

Здесь $|\cdot|$ - количество слов в строке, $\#A$ - число элементов в множестве $A$, $w \in s$ означает, что слово $w$ содержится в тексте $s$.

Если отношение $P / L$ больше 0.8, то функция должна вернуть True; иначе False.

Продемонстрируйте работу вашей функции на примере описаний двух рецептов с ID 135488 и 851934 (ID рецепта - это число, стоящее в конце url рецепта). Выведите на экран описания этих рецептов и результат работы функции.

In [9]:
def is_plagiarism(s1: str, s2: str) -> bool:
    s1 = word_tokenize(s1)
    s2 = word_tokenize(s2)
    L = max(len(s1), len(s2))
    P = 0
    for word in s1:
        for word_2 in s2:
            if nltk.edit_distance(word, word_2) < 2:
                P += 1
    # print(P / L)
    return (P / L) > 0.8



In [10]:
# Рецепты с нужным нам ID

recipes[(recipes["url"].str.endswith("851934/")) | (recipes["url"].str.endswith("135488/"))]

,url,name,ingredients,description
958,https://www.povarenok.ru/recipes/show/135488/,Паштет сало-авокадо в хлебных орешках,"{'Сало': '100 г', 'Соль': '1/3 ч. л.', 'Чеснок...",прекрасной закуской к крепким напиткам на фурш...
1473,https://www.povarenok.ru/recipes/show/851934/,Паштет из сала и авокадов хлебных орешках,"{'Сало': '100 г', 'Соль': '1/3 ч. л.', 'Чеснок...",замечательной закуской к напиткам на фуршетном...


In [11]:
is_plagiarism(recipes[recipes["url"].str.endswith("135488/")]["description"].values[0], recipes[recipes["url"].str.endswith("851934/")]["description"].values[0])

True

### Стемминг, лемматизация

4\. На основе набора слов из задания 2 создайте `pd.DataFrame` со столбцами `word`, `stemmed_word` и `normalized_word`. В столбец `stemmed_word` поместите версию слова после проведения процедуры стемминга; в столбец `normalized_word` поместите версию слова после проведения процедуры лемматизации. Столбец `word` укажите в качестве индекса. 

Для стемминга можно воспользоваться `SnowballStemmer` из `nltk`, для лемматизации слов - пакетом `pymorphy2`. Сравните результаты стемминга и лемматизации. Поясните на примере одной из строк получившегося фрейма (в виде текстового комментария), в чем разница между двумя этими подходами. 

In [12]:
morph = pymorphy2.MorphAnalyzer()
stemmer = SnowballStemmer("russian")
stem_lem_df = pd.DataFrame(columns=['word', 'stemmed_word', 'normalized_word'])
stem_lem_df['word'] = unique_words
stem_lem_df['stemmed_word'] = [stemmer.stem(word) for word in stem_lem_df['word']]
stem_lem_df['normalized_word'] = [morph.parse(word)[0].normalized.word for word in stem_lem_df['word']]
stem_lem_df = stem_lem_df.set_index('word')
stem_lem_df

,stemmed_word,normalized_word
word,,
серым,сер,серый
светланка,светланк,светланка
осталась,оста,остаться
случиться,случ,случиться
овсяными,овсян,овсяный
...,...,...
консистенцией,консистенц,консистенция
фунчозы,фунчоз,фунчоз
легкая,легк,лёгкий


5\. Добавьте в таблицу `recipes` столбец `description_no_stopwords`, в котором содержится текст описания рецепта после удаления из него стоп-слов. Посчитайте и выведите на экран долю стоп-слов среди общего количества слов. Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [13]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /home/noble6/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
stops = stopwords.words('russian')
pattern = r'\b(?:{})\b'.format('|'.join(stops))
recipes['description_no_stopwords'] = recipes['description'].str.replace(pattern, '')

/tmp/ipykernel_366420/1450924871.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  recipes['description_no_stopwords'] = recipes['description'].str.replace(pattern, '')


In [15]:
from collections import Counter


word_counts_before = Counter(word_tokenize(' '.join(recipes['description'])))
count_before, count_stops = sum(word_counts_before.values()), sum([word_counts_before[x] for x in stops])
count_before, count_stops, count_stops / count_before

(102978, 33223, 0.322622307677368)

In [16]:
word_counts_after = Counter(word_tokenize(' '.join(recipes['description_no_stopwords'])))
word_count_before = word_counts_before.most_common(10)
word_count_after = word_counts_after.most_common(10)
sum(word_counts_after.values()), count_before - count_stops

(69755, 69755)

In [17]:
word_count_before

[('и', 5054),
 ('в', 2584),
 ('с', 1934),
 ('на', 1655),
 ('очень', 1607),
 ('не', 1517),
 ('из', 1006),
 ('я', 979),
 ('рецепт', 869),
 ('а', 863)]

In [18]:
word_count_after

[('очень', 1607),
 ('рецепт', 869),
 ('это', 734),
 ('блюдо', 524),
 ('вкусный', 461),
 ('просто', 436),
 ('вкусно', 375),
 ('приготовить', 344),
 ('вкус', 324),
 ('салат', 313)]

### Векторное представление текста

6\. Выберите случайным образом 5 рецептов из набора данных, в названии которых есть слово "оладьи" (без учета регистра). Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`. На основе полученных векторов создайте `pd.DataFrame`, в котором названия колонок соответствуют словам из словаря объекта-векторизатора. 

Примечание: обратите внимание на порядок слов при создании колонок.

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

olad = recipes[recipes['name'].str.contains('оладьи')]
sample = olad.sample(n=5)
sample

def get_dataframe_vect(sent_words):
    cv = TfidfVectorizer()
    cv.fit(sent_words)
    return pd.DataFrame([{elem: cv.transform(sent_words).toarray()[0][cv.vocabulary_[elem]] for elem in cv.vocabulary_}])

get_dataframe_vect(nltk.sent_tokenize(sample['description'].values[0]))

,супер,быстрый,вариант,завтрака,это,даже,быстрее,чем,сварить,овсянку
0,0.316228,0.316228,0.316228,0.316228,0.316228,0.316228,0.316228,0.316228,0.316228,0.316228


7\. Вычислите близость между каждой парой рецептов, выбранных в задании 6, используя косинусное расстояние (можно воспользоваться функциями из любого пакета: `scipy`, `scikit-learn` или реализовать функцию самому). Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

Примечание: обратите внимание, что $d_{cosine}(x, x) = 0$

In [25]:
from sklearn.metrics.pairwise import cosine_similarity

random_descriptions = [sample.iloc[i]['description'] for i in range(5)]
vectorizer_vector = CountVectorizer().fit_transform(random_descriptions).toarray()
csim = cosine_similarity(vectorizer_vector)
csim



array([[1.        , 0.        , 0.        , 0.        , 0.03834825],
       [0.        , 1.        , 0.09682458, 0.06804138, 0.07426107],
       [0.        , 0.09682458, 1.        , 0.08784105, 0.05752237],
       [0.        , 0.06804138, 0.08784105, 1.        , 0.17516462],
       [0.03834825, 0.07426107, 0.05752237, 0.17516462, 1.        ]])

In [30]:
df = pd.DataFrame(csim, columns=sample['name'].values, index=sample['name'].values)
for i in range(5):
    df.iloc[i][i] = 0
df

,Бананово-кукурузные оладьи,Картофельно-творожные оладьи,"Картофельные оладьи с соусом ""Весна""",Голландские лимонные оладьи,Куриные оладьи с сыром
Бананово-кукурузные оладьи,0.000000,0.000000,0.000000,0.000000,0.038348
Картофельно-творожные оладьи,0.000000,0.000000,0.096825,0.068041,0.074261
"Картофельные оладьи с соусом ""Весна""",0.000000,0.096825,0.000000,0.087841,0.057522
Голландские лимонные оладьи,0.000000,0.068041,0.087841,0.000000,0.175165
Куриные оладьи с сыром,0.038348,0.074261,0.057522,0.175165,0.000000


8\. Напишите функцию, которая принимает на вход `pd.DataFrame`, полученный в задании 7, и возвращает в виде кортежа названия двух различных рецептов, которые являются наиболее похожими. Прокомментируйте результат (в виде текстового комментария). Для объяснения результата сравните слова в описаниях двух этих отзывов.

In [26]:
def find_closest(sim_df: pd.DataFrame) -> tuple:
    pass

In [31]:
def find_closest(sim_df: pd.DataFrame) -> tuple:
    print(sim_df[sim_df == sim_df.max().max()].stack().values[0])
    return sim_df[sim_df == sim_df.max().max()].stack().index.tolist()[0]
find_closest(df)



0.175164618081796


('Голландские лимонные оладьи', 'Куриные оладьи с сыром')

In [35]:
# Оладьи
print(sample[sample['name'] == find_closest(df)[0]]['description'].values[0])
print(sample[sample['name'] == find_closest(df)[1]]['description'].values[0])

# Расстояние - 17%, оба рецепта оладьи

0.175164618081796
  голландские оладьи особенность которых заключается в чуть более шарообразной форме в отличие от наших пышных но просто круглых оладий готовятся очень быстро с использованием дрожжей обжариваются на сковороде как на специальной с углублениями так и на совершенно обыкновенной рецепт который я нашла интересен наличием лимонного сока в тесте который придает оладушкам желтый оттенок и тем что при приготовлении требуется совсем небольшое количество сахара есть повод достать мед варенье шоколадный сироп и подавать голландские оладьи к завтраку с чемто таким сладким
0.175164618081796
лет двадцать назад покупая сыр наткнулась на рекламный листок к этому сыру и внутри был рецепт оладьев с простым составом и соотношением продуктов решила попробовать и в результате вот уже много лет это любимое блюдо которое потом разошлось по всем родственникам и друзьям оладушки получаются очень нежными с пикантным вкусом хороши как с пылужару так и холодными на бутерброд
все ингредиенты указ